<p class="pull-left">
ANLT 212 - Analytics Computing
</p>
<div class="pull-right">
<ul class="list-inline">
Copyright © Dana Nehoran 2017
</ul>
</div>

<div style="background-color: #A378AF; border:1px solid #000; margin-left: 2px; margin-right: 2px; 
padding-bottom: 2px; padding-left: 2px; padding-right: 1px; padding-top: 10px;">
<center><b><font size="6">Group Project 2 - Python Project</font></b></center>
</div>

Student Names:

<center><font size = 10>Tyler Norlund & Edwin Ramirez</font></center>

<blockquote style="border: 2px solid #666; padding: 10px; background-color: #acc;"><b>Question 1:</b> Project Definition</blockquote> 

Select a dataset from the UCI Machine Learning Repository 
http://archive.ics.uci.edu/ml/datasets.html
Your dataset should be different from the one utilized on project 1. 

Describe your stakeholder. Who is the potential sponsor of your project? For example: a real estate investor, the police department, a city mayor, etc. Explain what is the objective of this project and its justification. What are the findings so far, and what is your objective.

Note: <br>
If you want to select a dataset from a different public repository, you can.


<b><font color="blue" size = 4>a)</font> Data Selection</b>: Explain why this data was selected. What are the overall characteristics of the data.

<span style="background-color: #FFFF00">Your answer here</span>

In this project, we use the [Spotify API](https://developer.spotify.com/). This was done in python using the [Spotipy](https://spotipy.readthedocs.io/en/latest/#) python library. Using python, we obtained Spotify's top playlists and stored each playlist in a CSV format.

In [6]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from pandas import DataFrame
from os import listdir
from os.path import isfile, join


playlists = [f for f in listdir('./playlists/') if isfile(join('./playlists/', f))]
for playlist in playlists:
    print(playlist)

Everyday Favorites
Rhythm & Dance
LA Pops
I Love My '90s Hip-Hop
Doo-Wop Dee Doo
'90s Pop Rock Essentials
Autumn Acoustic
Latin Pride
Happy Hits!
Country's Greatest Hits_  The '60s
If It Wasn't For Gucci
Urban Poet
All Out 00s
Viva Latino
Energizing Classics
Country's Greatest Hits_  The '80s
Have a Great Day!
NIKE RUNNING FASTEST MILE
Calm Vibes
El Bloque
Love, Sex, & Water
Latina Power_ Jackie Cruz
You & Me
I Love My 80's Rollerdisco
Get Your Body Right
No Wahala
Crowd Pleasers
Hot Rhythmic
Sad Vibe
State of Mind
All Out 60s
70s Rock Anthems
Afropop.csv
Confidence Boost
RapCaviar
2000s Smash Hits
Tear Drop
Made in Chile
Warrior Workout
Mind Right
Spin Fit
Cloud Rap
Chinese Hip-Hop Stars
One-Hit Wonders
Indie Workout
Fresh Finds_ Poptronix
We Everywhere
Bodega Sounds
Lofi Hip-Hop
Mood Booster
Running & Workout
Feeling Accomplished
POLLEN
Way Back Workout
Beats & Rhymes
Spin Class
The Piano Lounge
Global X
Footwork Fever
Tender
Big Gains Workout
50 Latin Classics
Rockin' Vibes
Born To 

<b><font color="blue" size = 4>b)</font> Stakeholder</b>: Describe your stakeholder. Who is the potential sponsor of your project? For example: a real estate investor, the police department, a city mayor, etc. 

<span style="background-color: #FFFF00">Your answer here</span>

In this project, Spotify and music producers would be the stakeholders.

<b><font color="blue" size = 4>c)</font> Objective</b>: Project objective, justification, expected outcome. How your results may impact your stake holders. Which decisions or changes will the stakeholder be able to make based on this project

<span style="background-color: #FFFF00">Your answer here</span>

In this project we will classify songs based on the genres the songs belong to. This would help spotify better understand the types of songs people listen to and give Sp

<b><font color="blue" size = 4>d)</font> Background Research</b>: List here all other studies related to clustering published with the same dataset, and how your proposed study is different from them.

If no other clustering studies were conducted with the same dataset, you should specify: "No other studies available"

<span style="background-color: #FFFF00">Your answer here</span>

<blockquote style="border: 2px solid #666; padding: 10px; background-color: #acc;"><b>Question 2:</b> Exploratory Data Analysis</blockquote> 

Create some exploratory analysis on your data using core Python functions and visualizations. Cluster the information into different groups to explore the possibilities for your proposed project. Summarize your data and conduct some statistics. Explain your findings in English.

<b><font color="blue" size = 4>a)</font> Data Summary</b>: Show summary information of the different variables. Select the columns you are interested in. Explain each column, its range and purpose

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>b)</font> Data Cleaning</b>: Clean the data, removing rows and columns that have no useful information or no information at all

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>c)</font> Adaptation</b>: Create at least two additional columns that are necessary for your study. They can be calculated columns or aggregated columns.

<span style="background-color: #FFFF00">Your answer here</span>

<blockquote style="border: 2px solid #666; padding: 10px; background-color: #acc;"><b>Question 3:</b> Visualization with MatPlotLib</blockquote> 

<b><font color="blue" size = 4>a)</font> Descriptive Analytics</b>: Create at least 5 different (unique) visualizations that show different aspects of your data related to the research object of your project

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>b)</font> Outcome</b>: Explain in words the outcome of your descriptive analysis

<span style="background-color: #FFFF00">Your answer here</span>

<blockquote style="border: 2px solid #666; padding: 10px; background-color: #acc;"><b>Question 4:</b> Clustering</blockquote> 

Select a number of numerical columns to be used for your clustering algorithms. Explain the business logic of your clustering. Select expected number of clustering to explore.

<b><font color="blue" size = 4>a)</font>  Cluster Analysis</b>: Explain the business reason for your selection of the number of clusters.

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>b)</font> k-Means</b>: Cluster your data using k-means. Explain your results. Try two different schemas of clusters.

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>c)</font> Hierarchical Clustering</b>: Cluster your data using hierarchical clustering. Explain your results. Try two different schemas of clusters.

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>d)</font> kNN Clustering</b>: Use your results from the cluster analysis performed in a) or b) and create a short training set. Cluster your data using your training set on kNN. Explain your results.

<span style="background-color: #FFFF00">Your answer here</span>

<blockquote style="border: 2px solid #666; padding: 10px; background-color: #acc;"><b>Question 5:</b> Summary</blockquote> 

<b><font color="blue" size = 4>a)</font> Project Summary</b>: Write a few sentences about the result of this project. How can your stakeholder benefit from the results of your project? Which changes or adaptations can your stakehoders make now that he has your results?

<span style="background-color: #FFFF00">Your answer here</span>

<b><font color="blue" size = 4>b)</font> Project Report</b>: Write a formal report to your stakeholders with the summary of your report 

<span style="background-color: #FFFF00">Your answer here</span>